### HW 6

Dataset : Employee data

#### 1.Preprocess Test data and get predictions

In [42]:
#Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [43]:
data = pd.read_csv('employee.csv')
data.head(10)

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000
5,6,1/8/2017 12:58:49,United Kingdom,Full time,Developer,1.00000,No,38.0,0.0,Bachelors (4 years),No,No,5833.000000
6,7,1/9/2017 11:27:40,Canada,Full time,DBA,5.00000,No,40.0,0.0,Associates (2 years),Yes,Yes,7166.000000
7,8,12/13/2017 7:41:41,United States,Full time,DBA,0.00000,No,45.0,4.0,Bachelors (4 years),No,Yes,8750.000000
8,9,1/9/2017 19:37:30,United States,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
9,10,12/12/2017 11:44:16,New Zealand,Full time,DBA,5.00000,Yes,40.0,0.0,Bachelors (4 years),No,No,7166.000000


In [44]:
# Dimensions of training data
data.shape

(4277, 13)

In [45]:
data = data.drop(columns = ['id','timestamp','country'])

In [46]:
data.columns
data.isnull().sum()

employment_status                 0
job_title                         0
job_years                         0
is_manager                        0
hours_per_week                   17
telecommute_days_per_week        11
education                         0
is_education_computer_related    16
certifications                    0
salary                            0
dtype: int64

In [47]:
# replace NANs
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()
data = data.dropna()

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


In [49]:
# create another copy of dataset and append encoded features to it
data_train = data.copy()
data_train.head()

# select categorical features
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols

#Encoding binary variables
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)
    
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)
final_data.head(10)

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,...,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.78393,1,40.0,0.0,0,7187.743094,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,5.00000,0,40.0,5.0,0,10000.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,1.00000,0,40.0,0.0,1,7000.000000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,1.00000,0,40.0,2.0,0,8333.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3.00000,0,40.0,2.0,1,7137.000000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
5,1.00000,0,38.0,0.0,0,5833.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,5.00000,0,40.0,0.0,1,7166.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0.00000,0,45.0,4.0,1,8750.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,4.78393,1,40.0,0.0,0,7187.743094,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
9,5.00000,1,40.0,0.0,0,7166.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [50]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


In [51]:
# select numerical features
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']

# Apply standard scaling on numeric data
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,...,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3571,0.252676,0,-0.498388,1.807343,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3386,0.252676,0,-0.498388,-0.623129,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
1210,-0.273580,0,-0.326315,-0.623129,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
2145,0.109554,1,0.458248,-0.623129,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1803,0.252676,0,2.353552,-0.623129,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,0.252676,0,-0.498388,-0.623129,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
872,0.252676,0,-0.498388,1.199725,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3849,-0.757004,0,-0.498388,-0.015511,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
332,0.071353,0,0.137103,-0.623129,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1


In [52]:
#Fitting a Linear Regression Model
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [53]:
reg.predict(X_train)

array([7633.37291296, 7061.72024   , 6614.21551833, ..., 7019.68848858,
       7149.56658336, 7066.81932857])

In [58]:
y_pred = reg.predict(X_test)
y_pred

array([15882.37575116, 15939.16752744, 16931.57635248, 14920.26731667,
       15429.55833836, 15074.88232736, 15036.87486611, 15922.2348033 ,
       15966.69151864, 16039.74061246, 16114.54275271, 15784.70459278,
       15532.74006255, 17010.6605067 , 16267.25849617, 17075.84888757,
       14844.60245774, 15750.27276686, 14287.81821964, 15249.43360356,
       16432.92112563, 15443.18833439, 16912.87388962, 16597.87661731,
       17449.19012592, 16207.9360716 , 15975.34142749, 14983.22460176,
       15469.78277747, 15638.28003915, 15600.88788058, 15557.77939787,
       17020.34046041, 16397.49834754, 14372.18303158, 16111.74082504,
       15715.91188282, 16144.7969948 , 16556.77329108, 16780.41192596,
       15868.18166789, 15125.84447648, 16968.72812896, 15608.99778696,
       14987.27984474, 18453.18785149, 16153.24692258, 14986.28562796,
       17549.85569366, 14802.43052931, 15371.74764521, 15282.82819408,
       15219.38182496, 15668.80383347, 15032.08309698, 15794.27574688,
      

2.Compute Mean Absolute Error, Mean Square error for test data.

In [59]:
# Normalized MSE (Dividing by mean)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test, y_pred)/np.mean(y_test)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

Mean Absolute Error: 8865.251617973418
Mean Squared Error: 11419.363237802598


3.Optional : Read about Ridge and Lasso Regression, implement them and compute evaluation metrics. Do they perform better than Linear Regression?

In [61]:
from sklearn.linear_model import Ridge, Lasso

In [65]:
# Train Ridge and Lasso Regression models
ridge_model = Ridge(alpha=1.0)  

ridge_model.fit(X_train, y_train)

#predictions on the test set
ridge_predictions = ridge_model.predict(X_test)

#Compute evaluation metrics (MAE and MSE)
ridge_mae = mean_absolute_error(y_test, ridge_predictions)
ridge_mse = mean_squared_error(y_test, ridge_predictions)/np.mean(y_test)

print("Ridge Regression Evaluation Metrics:")
print("Mean Absolute Error (MAE):", ridge_mae)
print("Mean Squared Error (MSE):", ridge_mse)

Ridge Regression Evaluation Metrics:
Mean Absolute Error (MAE): 8818.50206826804
Mean Squared Error (MSE): 11301.697420397326


In [66]:
lasso_model = Lasso(alpha=1.0) 

lasso_model.fit(X_train, y_train)

lasso_predictions = lasso_model.predict(X_test)

lasso_mae = mean_absolute_error(y_test, lasso_predictions)
lasso_mse = mean_squared_error(y_test, lasso_predictions)/np.mean(y_test)

print("Lasso Regression Evaluation Metrics:")
print("Mean Absolute Error (MAE):", lasso_mae)
print("Mean Squared Error (MSE):", lasso_mse)

Lasso Regression Evaluation Metrics:
Mean Absolute Error (MAE): 8726.180072087904
Mean Squared Error (MSE): 11071.134282134763


#### Compare to the MAE, MSE results from linear regression, Ridge and Lasso Regression preform better than Linear Regression